**Step 1**

Import the required Python libraries

Include **app_id** & **app_name** from the iOS App Store URL

Set number of reviews needed with **how_many**

Set minimum date for reviews with **after**

In [14]:
import pandas as pd
import numpy as np
import json
import datetime

from app_store_scraper import AppStore
MSA_iOS = AppStore(country='us', app_name='my-spectrum', app_id = '942608209')

MSA_iOS.review(how_many=2000, after=datetime.datetime(2023, 1, 1))

2023-03-29 21:44:37,401 [INFO] Base - Initialised: AppStore('us', 'my-spectrum', 942608209)
2023-03-29 21:44:37,402 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/us/app/my-spectrum/id942608209
2023-03-29 21:44:42,450 [INFO] Base - [id:942608209] Fetched 51 reviews (51 fetched in total)
2023-03-29 21:44:47,687 [INFO] Base - [id:942608209] Fetched 99 reviews (99 fetched in total)
2023-03-29 21:44:52,956 [INFO] Base - [id:942608209] Fetched 133 reviews (133 fetched in total)
2023-03-29 21:45:12,699 [ERROR] Base - Something went wrong: HTTPSConnectionPool(host='amp-api.apps.apple.com', port=443): Max retries exceeded with url: /v1/catalog/us/apps/942608209/reviews?l=en-GB&offset=2000&limit=20&platform=web&additionalPlatforms=appletv%2Cipad%2Ciphone%2Cmac (Caused by ResponseError('too many 429 error responses'))
2023-03-29 21:45:12,702 [INFO] Base - [id:942608209] Fetched 146 reviews (146 fetched in total)


**Step 2**

Add the pulled data to a dataframe

Convert the JSON data to CSV

Note that the developer response information is stored as a series

In [19]:
MSA_iOSdf = pd.DataFrame(np.array(MSA_iOS.reviews),columns=['review'])
MSA_iOSdf2 = MSA_iOSdf.join(pd.DataFrame(MSA_iOSdf.pop('review').tolist()))
MSA_iOSdf2.head()

,date,developerResponse,review,rating,isEdited,title,userName
0,2023-03-01 16:55:19,"{'id': 35171306, 'body': 'We're sorry that you...",I’ve had spectrum/time Warner Wi-Fi for over d...,3,False,Spectrum Cellular,BRANDON. LOVE
1,2023-03-03 00:26:01,NaN,"First of all, if you’re forced to get Spectrum...",1,False,Spectrum app and service is a scam,samross19
2,2023-02-03 03:35:09,"{'id': 34672035, 'body': 'We're sorry that you...",I’ve spent 26 hours trying to install all of t...,1,False,WORST COMPANY EVER!!!,jwhall640
3,2023-01-08 06:20:22,NaN,Spectrum is really a poor quality service that...,1,False,Specters really a poor company,roy william g
4,2023-01-05 04:38:41,NaN,First off I was on the phone line with a agent...,1,False,Worse customer service,Precise23


**Step 3**

Split out the ***developerResponse*** data from the dataset as ***devdf***

Change the series data in ***developerResponse*** into a Dataframe

In [20]:
df = MSA_iOSdf2.developerResponse
devdf = pd.json_normalize(df)
devdf.head()

,id,body,modified
0,35171306.0,We're sorry that you're having issues with you...,2023-03-02T19:50:01Z
1,NaN,NaN,NaN
2,34672035.0,We're sorry that you're having issues with you...,2023-02-04T11:32:58Z
3,NaN,NaN,NaN
4,NaN,NaN,NaN


**Step 4**

Add an index column to both dataframes, ***devdf*** & ***MSA_iOSdf2***

As the dataset was split without being reordered, the index column can be used to merge both dataframes

Rearrange the columns using **iloc** as ***combinedReorder***

In [21]:
devdf['index']=devdf.index
MSA_iOSdf2['index']=MSA_iOSdf2.index
combined = devdf.merge(MSA_iOSdf2, how = 'outer', right_on='index', left_on='index')
combined.rename(columns={'body':'dev response','modified':'dev response date','date':'review date','title':'review title'}, inplace=True)
combinedReorder = combined.iloc[:,[3,0,2,1,4,9,6,7,8,10,5]]

**Step 5**

Drop the ***developerResponse*** & ***index*** column from the dataset

The ***devdf*** dataframe already pulled and formatted the required data from ***developerResponse*** in Step 3

Export the ***CombinedFinal*** dataframe to a CSV file

In [22]:
combinedFinal = combinedReorder.drop(columns=["developerResponse","index"])
combinedFinal.to_csv('MSA_iOS_Reviews.csv')